In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
from scipy import interpolate
from scipy.optimize import curve_fit

In [2]:
R = 8.314 # Sets R(gas constant) value in [J/(mol-K)]

In [11]:
def read_ads_data(mof, temperature): # MOF name and temp data was recorded 
    ads_data = pd.read_csv('data/{}_{}C.csv'.format(mof, temperature)) # Reads the isotherm 
    
    ads_data['P/P_0'] = ads_data['RH[%]'] / 100 # Converts the relative humidity to P/P_0
    
    ads_data["A"] = -R * (temperature+273.15) * np.log(ads_data['P/P_0']) # Calculates Polanyi adsorption potential for every point on the graph 

    
    ads_data = ads_data.drop(columns=['RH[%]']) # Gets rid of the Humidity column 
    return ads_data

In [12]:
# Reads all of the different temp MOF 801 isotherms 
ads_data_15 = read_ads_data('MOF_801',15)
ads_data_25 = read_ads_data('MOF_801',25)
ads_data_45 = read_ads_data('MOF_801',45)
ads_data_65 = read_ads_data('MOF_801',65)
ads_data_85 = read_ads_data('MOF_801',85)
ads_data_105 = read_ads_data('MOF_801',105)
 
combined_df = pd.concat([ads_data_15, ads_data_25, ads_data_45, ads_data_65, ads_data_85,ads_data_105]) # Combines all MOF 801 data into a single dataframe 

In [13]:
combined_df_sorted = combined_df.sort_values('A') # Sorts the MOF 801 data by A values so it can be plotted and the curve can be calculated 

In [14]:
# NOT HAPPY WITH THIS STEP 
n_of_A = interpolate.CubicSpline(combined_df_sorted['A'], combined_df_sorted['Water Uptake [kg kg-1]']) # Finds the equation of the characteristic curve 

In [15]:
#RH in % here 
def calculate_water(T,P_over_P_0):
    A = -R * (T+273.15) * np.log(P_over_P_0)
    Rate_H2O = n_of_A(A)
    
    return(Rate_H2O.item())

In [16]:
#Source for weather data 
#https://weathersight.io/timeseries
#For Phoenix from 2020-2021
df_weather = pd.read_csv('data/Phoenix_weather_edited_final.csv')

In [17]:
#Need a function that pulls the weekly data out of the dataframe 
def weekly_data(week):
    
    df_week = df_weather['Week'] == week
    T_high = df_weather[df_week]['T_max_mean[C]'].item()
    T_low = df_weather[df_week]['T_min_mean[C]'].item()
    Dewp_high = df_weather[df_week]['Max_dewp[C]'].item()
    Dewp_low = df_weather[df_week]['Min_dewp[C]'].item()
    
    return T_high, T_low, Dewp_high, Dewp_low                        

In [7]:
#Find the saturated vapor pressure from temp and dewp 
#source for formula 
#https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf
def pressure_calculations(T,T_dew):
    
    P_0 = 6.11 * 10**((7.5*T)/(237.3+T))
    P = 6.11 * 10**((7.5*T_dew)/(237.3+T_dew))
    P_over_P_0 = P/P_0
    
    return P_over_P_0

In [8]:
#Find mass of water depending on daily high and low, and mass MOF 
def water_daily(week,kg_MOF):
    
    T_high, T_low, Dewp_high, Dewp_low = weekly_data(week)
    
    P_over_P_0_high = pressure_calculations(T_high, Dewp_high)
    
    P_over_P_0_low = pressure_calculations(T_low, Dewp_low)
    
    A_low = -R * (T_low+273.15) * np.log(P_over_P_0_high/100)
    
    A_high = -R * (T_high+273.15) * np.log(P_over_P_0_low/100)
    
    high_water = calculate_water(T_high, P_over_P_0_high)
    
    low_water = (calculate_water(T_low, P_over_P_0_low))
    
    water_rate = low_water - high_water
    
    water_mass = water_rate * kg_MOF
    
    print('The mass of water collected by the MOF per day during week', week, 'is',round(water_mass,4),'[kg/day]')

    return water_mass

In [ ]:
water_daily(11,1)

The mass of water collected by the MOF per day during week 11 is 0.044 [kg/day]


0.04395230171836606